# Supervised Learning
In diesem Jupyter Notebook wird durch die Entwicklung und Anpassung eines Modells für maschinelles (überwachtes/supervised) Lernen geführt . Beginnend mit der Analyse der Daten und der Festlegung relevanter Kriterien, leiten wir durch die Auswahl eines geeigneten Modells. User können das Modell ohne Programmierkenntnisse optimieren; angeleitete, geringfügige Anpassungen sind möglich.

## Teil 1: Datenexploration
Ziele:  

    Verständnis der Datenstruktur und der Features
    Visualisierung der Beziehungen zwischen verschiedenen Features und den Zielklassen  

Aufgaben:  

    Daten laden, analysieren und mit Boxplots und Pairplots visualisieren

Hier analysieren wir einen Datensatz, der chemische Eigenschaften verschiedener Weinsorten umfasst. Die Variable „target“ mit den Werten 0, 1 und 2 repräsentiert drei unterschiedliche Weinkultivare aus einer italienischen Region, wobei jedes Ziel einem anderen Weinproduzenten oder einer spezifischen Weinsorte zugeordnet ist. Die Herausforderung liegt darin, basierend auf der chemischen Zusammensetzung des Weins – beispielsweise Alkohol-, Magnesiumgehalt und Flavonoiden – die entsprechende Weinsorte oder den Produzenten (das Ziel) zu identifizieren. Diese chemischen Charakteristika unterscheiden sich merklich zwischen den Weinsorten, und das Ziel dieses Projekts ist es, ein Modell zu entwickeln, das zuverlässig zwischen den drei Weinsorten oder -herstellern differenzieren kann.

Im Folgenden werden die nötigen Bibliotheken importiert und der Wein-Datensatz geladen. Dieser ist, genau wie der Iris-Datensatz aus unserem Beispiel in `example_supervised.ipynb`, sehr gut geeignet, um ML-Konzepte zu verstehen und zu testen, hat aber deutlich mehr Features.


In [ ]:
# run the cell
# Import der Bibliotheken
from sklearn.datasets import load_wine
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
#plt.style.use('dark_background')

# Datensatz laden
wine_data = load_wine()
X = wine_data.data
y = wine_data.target

# Ausgabe der Merkmalsnamen
print(wine_data.feature_names)

# Erstellung einer Liste mit den Übersetzten Merkmalsnamen
angepasste_feature_namen = [
    'Alkohol',
    'Apfelsäure',
    'Asche',
    'Aschealkalität',
    'Magnesium',
    'Gesamtphenole',
    'Flavonoide',
    'Nichtflavonoide_Phenole',
    'Proanthocyanidine',
    'Farbintensität',
    'Farbton',
    'Optische_Dichte',  # bei 280 und 315 nm von verdünnten Wein
    'Prolin'
]

# Daten in einen DataFrame umwandeln und für die Spaltenbeschriftungen die übersetzten merkmanlsnamen verwenden
wine_df = pd.DataFrame(X, columns=angepasste_feature_namen)
wine_df['target'] = y

# Einfache Datenanalyse
print(wine_df.describe())


Mit `.shape` können wir die Größe des Datensatzes ermitteln:

In [ ]:
# run the cell
wine_df.shape

Der Datensatz besteht aus 178 Zeilen und 14 Spalten. Davon ist eine Spalte das Ziel und 13 die Features. 
Um die Daten besser zu verstehen, werden im Folgenden Boxplots und Pairplots der einzelnen Features erstellt. Das hilft herauszufinden, ob die Weine Eigenschaften haben, über die sie sich gut klassifizieren lassen.

In [ ]:
# run the cell
# Boxplots für jedes Feature
plt.figure(figsize=(12,10))
for i, col in enumerate(wine_df.columns[:-1]):
    plt.subplot(4, 4, i + 1)
    sns.boxplot(x='target', y=col, data=wine_df)
    plt.tight_layout()
plt.show()

Das Bild zeigt eine Sammlung von Boxplots, jeweils für verschiedene chemische Merkmale des Wein-Datensatzes, aufgeteilt nach den drei Klassen (target 0, 1 und 2). Die target-Variable repräsentiert die drei verschiedenen Weinsorten oder -hersteller. Für jedes Feature gibt es drei Boxplots, einen für jede Weinsorte oder jeden Hersteller (target = 0, target = 1, target = 2).
Was können wir beobachten?  
**Flavonoide:** Die Verteilung von Flavonoiden scheint bei den drei Klassen unterschiedlich zu sein. Insbesondere target 2 zeigt niedrigere Werte im Vergleich zu target 0 und target 1. Dies bedeutet, dass Flavonoide ein gutes Unterscheidungsmerkmal zwischen den Klassen sein könnten.

**Farbintensität:** Es gibt deutliche Unterschiede in der Farbintensität zwischen den Klassen, wobei target 2 die höchste Varianz aufweist und target 0 und target 1 eher niedrigere Werte zeigen.  

**Alkohol:** Auch der Alkoholgehalt unterscheidet sich zwischen den Klassen. Target 0 scheint im Durchschnitt höhere Alkoholwerte zu haben als target 1 und target 2.

**Farbton und optische Dichte bei Verdünnung** Hier sieht man, dass sich target 2 sehr deutlich von den anderen beiden Weinherstellern oder -sorten unterscheidet. 

Für die Klassifikation sind also Merkmale besonders wertvoll, die eine gute Trennung zwischen den Klassen ermöglichen. In diesem Fall scheinen `Farbton` und `Optische_Dichte` starke Features für die Modellauswahl zu sein, weil sie eine Klasse klar von den anderen abgrenzen.

Wenn wir also überlegen, welche Features für ein Klassifikationsmodell ausgewählt werden sollten, wollen wir nicht nur Merkmale, die in sich selbst aussagekräftig sind, sondern auch solche, die in Kombination mit anderen Merkmalen gut funktionieren. Hier kommen Pairplots ins Spiel.

Ein Pairplot visualisiert:

    Die Verteilung jedes Merkmals entlang der Diagonale.
    Die bivariaten Beziehungen zwischen jedem Merkmalspaar in den anderen Zellen.


In [ ]:
# run the cell
# Pairplot für ausgewählte Features - kann angepasst werden 
sns.pairplot(wine_df, vars=['Alkohol', 'Apfelsäure', 'Flavonoide', 'Farbintensität','Farbton','Optische_Dichte'], hue='target', palette='Set2')
plt.show()


Das Pairplot-Diagramm zeigt eine Matrix von Beziehungen zwischen den ausgewählten Merkmalen des Wein-Datensatzes, aufgeteilt nach den Zielklassen. 

**Aufgabe:** Was kann beobachtet werden? Achte auch darauf, welche Features einen linearen Zusammenhang aufweisen (Stichwort Multikollinearität).

Schreibe deine Beobachtungen hier auf (Doppelklick in die Zelle):
    -  
    -  
    -  
    -  
    -  
    -  
    -  



## Teil 2: Feature-Auswahl und Modelltraining
Ziele:  

    Verständnis der Bedeutung von Feature-Auswahl und wie diese die Modellleistung beeinflussen kann
    Praktische Erfahrung im Training eines Modells und Anpassung von Hyperparametern  

Aufgaben:  

    Diskussion über Feature-Auswahl:
        Warum ist es wichtig, Features sorgfältig auszuwählen. 

    Auswahl von Features für das Modell:
        Basierend auf den Visualisierungen, entscheide, welche Features ins Modell aufgenommen werden sollen. Die Entscheidung sollte auf Faktoren wie der Verteilung der Daten, der Trennung zwischen den Klassen und der Vermeidung von Multikollinearität basieren.

    Modelltraining und -evaluation:
       Klassifikationsmodell mit den ausgewählten Features trainieren. Experimentiert werden kann mit unterschiedlichen Werten von k für den k-NN-Algorithmus oder die Features die ins Modell gegeben werden. Beobachte die Auswirkungen auf die Modellleistung.


Wir beginnen mit dem Trainieren eines Modells und werden zuerst alle Features für das Training nutzen. Genauso wie im Beispiel mit dem Iris-Datensatz teilen wir unsere Daten inin einen Trainings- und einen Testdatensatz auf. Danach wählen wir die 5 nächsten Nachbarn für die Klassifikation aus und gewichten die Daten erst einmal nicht. 


In [ ]:
# run the cell
# Datensatz aufteilen in Trainings- und Testdaten
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Modell initialisieren und trainieren
knn = KNeighborsClassifier(n_neighbors=5, weights='uniform')
knn.fit(X_train, y_train)

# Modell evaluieren
y_pred = knn.predict(X_test)
#print('Konfusionsmatrix:\n', confusion_matrix(y_test, y_pred))
print('Klassifikationsreport:\n', classification_report(y_test, y_pred))


**Was sagt der Klassifikationsreport aus:**  

•	**Support:** Die Anzahl der tatsächlichen Vorkommen jeder Klasse in den Testdaten. Zum Beispiel gab es 19 Instanzen von target = 0 im Testset.
•	**Precision:** Der Anteil der tatsächlichen Positiven an allen positiven Ergebnissen. Hier bedeutet eine Precision von 0.89 für target = 0, dass von allen Fällen, die das Modell als target = 0 klassifiziert hat, 89% tatsächlich target = 0 waren.  
•	**Recall:** zeigt, wie gut das Modell alle tatsächlichen positiven Fälle identifiziert. Ein Recall von 1.00 für target = 0 würde bedeuten, dass das Modell alle Weine von target = 0 korrekt identifiziert hat. Hier liegt der Recall bei 89%, d.h. von allen Weinen, die tatsächlich zu target = 0 gehören, hat das Modell 89% richtig als target = 0 klassifiziert, während 11% der tatsächlichen target = 0 Weine möglicherweise fälschlicherweise einer anderen Kategorie zugeordnet wurden.  
•	**F1-Score:** Das harmonische Mittel aus Precision und Recall. Ein F1-Score von 1.00 ist perfekt, während 0.00 das schlechteste Ergebnis ist. Für target = 2 hat das Modell einen F1-Score von 0.55, das ist nicht besonders gut.  
•	**Accuracy:** Der Anteil der Gesamtvorhersagen, die korrekt waren. Eine Accuracy von 0.74 bedeutet, dass 74% der Vorhersagen des Modells über alle Klassen korrekt waren.  
•	**Macro Avg:** Der ungewichtete Durchschnitt der Metriken für alle Klassen. Dies ist nützlich, wenn Klassen als gleich wichtig betrachtet werden.  
•	**Weighted Avg:** Der durchschnittliche Wert jeder Metrik, gewichtet nach der Anzahl der Instanzen in jeder Klasse. Dies berücksichtigt die Ungleichgewichte in den Klassengrößen.  

Die Modellgenauigkeit von 74% ist nicht herausragend und das ist auf jeden Fall ein Hinweis darauf, das hier noch einiges optimiert werden kann. Gerade die Werte für target = 2 sind schlecht, obwohl sich diese Klasse doch bei einigen Features deutlich von den anderen Beiden unterscheidet.

In der nächsten Zelle wird die Konfusionsmatrix geplottet, die noch einmal zeigt, wie sich die Vorhersagen verteilen.

In [ ]:
# run the cell
# Konfusionsmatrix erstellen
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Plot
fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', 
            xticklabels=wine_data.target_names, yticklabels=wine_data.target_names)
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix')

plt.show()

### Modellanpassungen

Als aller erstes ist es sinnvoll, die Anzahl der Features zu reduzieren. Mit Hilfe der Pair- und Boxplots konnte man schon sehr gut sehen, welche Features interessant sind und sich beesonders für die Klassifikation der Zielvariablen (targets) eignen.


In der folgenden Codezelle können in der Liste `selected_features` die gewählten Features eingetragen werden. Experimentiere mit verschiedenen Kombinationen, um zu sehen, wie sich die Änderungen auf den Klassifikationsreport auswirken und ob eine Verbesserung des Modells erkennbar ist.

**Fragestellungen:**  
- Wie verändern sich die Werte im Klassifikationsreport im Vergleich zu den ursprünglichen Einstellungen? Ist eine Verbesserung des Modells erkennbar?


In [ ]:
# hier Anpassungen für selected_features machen und Namen der gewünschten Merkmale eintragen
# Ausgewählte Features
# z.B. selected_features = ['Alkohol','Optische_Dichte',...]#
selected_features = []#

# Nur die ausgewählten Features für X verwenden
X_selected = wine_df[selected_features].values

# Datensatz aufteilen in Trainings- und Testdaten
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.3, random_state=42)

# Modell initialisieren und trainieren
# hier Anpassungen für n_neighbors und weights machen
knn = KNeighborsClassifier(n_neighbors=5,weights = 'uniform' )
#knn = KNeighborsClassifier(n_neighbors=7,weights = 'distance' )
knn.fit(X_train, y_train)

# Modell evaluieren
y_pred = knn.predict(X_test)
print('Konfusionsmatrix:\n', confusion_matrix(y_test, y_pred))
print('Klassifikationsreport:\n', classification_report(y_test, y_pred))


**Nächste Schritte:**

1. Ändere die Anzahl der Nachbarn (`n_neighbors`). Wir haben zuvor 5 Nachbarn betrachtet (`n_neighbors=5`). Experimentiere mit dieser Anzahl, um deren Einfluss auf das Modell zu verstehen.
2. Ändere `weights` von `uniform` zu `distance`. Beobachte, was mit dem Modell geschieht.

Diese Schritte gehören zum sogenannten **Hyperparameter-Tuning**, bei dem mit den Einstellungen des Modells experimentiert wird, um die beste Leistung zu erzielen.

**Was sind Hyperparameter?**  
Hyperparameter sind die Konfigurationseinstellungen, die vor dem Trainingsprozess eines maschinellen Lernmodells festgelegt werden. Sie steuern die Struktur des Modells (wie die Anzahl der Nachbarn in einem kNN-Modell) und wie das Modell trainiert wird (wie z.B. Lernrate in neuronalen Netzen). Im Gegensatz zu Modellparametern, die während des Trainings aus den Daten gelernt werden, werden Hyperparameter nicht vom Modell selbst angepasst. Das Tuning von Hyperparametern ist wichtig, um optimale Modellleistung zu erreichen.

In [ ]:
# kopiere den Code aus der zelle davor und ändere n_neighbors und oder weights





In [ ]:
# Konfusionsmatrix für Plot erstellen
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Plot
fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', 
            xticklabels=wine_data.target_names, yticklabels=wine_data.target_names)
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix')

plt.show()

Um die optimale Anzahl von `n_neighbors` für das Modell zu finden, kannst du eine Schleife nutzen, um verschiedene Werte für `k` systematisch zu testen. Durch die Auswertung der Ergebnisse für jedes `k` kannst du den optimalen Wert ermitteln. Für das training verwenden wir die Datensätze `X_train` und `y_train`, die basierend auf vorher festgelegten Kriterien erstellt wurden. Experimentiere mit diesem Ansatz, um den Wert für `k` zu finden, der die beste Modellleistung bietet.

In [ ]:
# run the cell
# Verschiedene Werte für k ausprobieren
k_values = range(1, 26)
scores = []

for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k, weights = 'distance')
    knn.fit(X_train, y_train)
    scores.append(knn.score(X_test, y_test))

# Plot der Modellleistung in Abhängigkeit von k
plt.figure(figsize=(10, 6))
plt.plot(k_values, scores, marker='o', linestyle='-', color='red')
plt.title('Modellleistung in Abhängigkeit von k')
plt.xlabel('k')
plt.ylabel('Genauigkeit')
plt.show()

## Teil 3: Reflexion und Diskussion 

Reflektiere über den Prozess der Feature-Auswahl, die Herausforderungen bei der Modellanpassung und die Bedeutung von Hyperparameter-Tunings
        
Präsentiert eure Ergebnisse vielleicht in Gruppen, erklärt eure Entscheidungen und wie diese die Modellleistung beeinflusst haben. Was hat gut funktioniert, was hat nicht so gut funktioniert?  






## Zusatz: Andere Modelle zur Klassifizierung von Daten

Neben den gängigen Klassifizierungsmodellen wie k-Nearest Neighbors (kNN) gibt es weitere Modelle, die für Klassifizierungsprobleme genutzt werden können. Drei Beispiele sind Entscheidungsbäume, Support Vector Machines (SVC) und Random Forests. Jedes dieser Modelle hat einzigartige Eigenschaften, die sie für bestimmte Arten von Datensätzen und Problemen geeignet machen:

- **Entscheidungsbäume (Decision Trees):** Einfach zu verstehen und zu interpretieren, Entscheidungsbäume spiegeln menschliche Entscheidungsfindung wider, indem sie Daten in einem baumähnlichen Modell von Entscheidungen und deren möglichen Konsequenzen strukturieren. [Mehr erfahren](https://scikit-learn.org/stable/modules/tree.html#tree).

- **Support Vector Machines (SVC):** Diese Modelle sind besonders geeignet für komplexe Klassifikationsprobleme mit klarer Trennung der Daten. Sie versuchen, eine Hyper-Ebene zu finden, die die verschiedenen Klassen gut trennt. [Mehr erfahren](https://scikit-learn.org/stable/modules/svm.html#svm-classification).

- **Random Forests:** Als Erweiterung der Entscheidungsbäume bauen Random Forests mehrere Entscheidungsbäume auf und fassen ihre Vorhersagen zusammen, um die Genauigkeit zu verbessern und Overfitting zu vermeiden. Sie sind robust gegenüber Overfitting und eignen sich gut für große Datensätze. [Mehr erfahren](https://scikit-learn.org/stable/modules/ensemble.html#forest).

In den folgenden Codezellen werden wir diese Modelle am Beispiel des Weindatensatzes testen, dafür wählen wir wieder die selektierten features wie zuvor. Es wird auch gezeigt, wie Hyperparameter dieser Modelle getuned werden können, um die beste Leistung zu erzielen. Wir werden das Hyperparameter-Tuning am Beispiel des Random Forest-Modells demonstrieren.


Entscheidungsbaum (Decision Tree)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Entscheidungsbaum Modell erstellen und trainieren
tree_model = DecisionTreeClassifier()
tree_model.fit(X_train, y_train)

# Vorhersagen und Konfusionsmatrix
y_pred = tree_model.predict(X_test)
conf_mat = confusion_matrix(y_test, y_pred)

# Konfusionsmatrix plotten
sns.heatmap(conf_mat, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

print(classification_report(y_test, y_pred))

Support Vector Machine

In [ ]:
from sklearn.svm import SVC

# Modell erstellen und trainieren
svm_model = SVC()
svm_model.fit(X_train, y_train)

# Vorhersagen und Konfusionsmatrix
y_pred = svm_model.predict(X_test)
conf_mat = confusion_matrix(y_test, y_pred)

# Konfusionsmatrix plotten
sns.heatmap(conf_mat, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

print(classification_report(y_test, y_pred))

Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Random Forest Modell erstellen und trainieren
forest_model = RandomForestClassifier(n_estimators=100)
forest_model.fit(X_train, y_train)

# Vorhersagen und Konfusionsmatrix
y_pred = forest_model.predict(X_test)
conf_mat = confusion_matrix(y_test, y_pred)

# Konfusionsmatrix plotten
sns.heatmap(conf_mat, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

print(classification_report(y_test, y_pred))

**Hyperparameter-Tuning für Random Forest**  

Wir erstellen ein Dictionairy an Hyperparametern, die mit RandomizedSearchCV das Modell mit den verschiedenen Parametern testen und uns am Ende die Besten ausgeben. Damit erstellen und trainieren wir dann das Modell.  

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

param_distributions = {
    'n_estimators': [200, 300],
    'max_features': ['log2'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2],
    'min_samples_leaf': [1, 2, 4],
}

random_search = RandomizedSearchCV(RandomForestClassifier(), param_distributions, n_iter=10, cv=5, scoring='accuracy')
random_search.fit(X_train, y_train)

print("Beste Parameter:", random_search.best_params_)

In [ ]:

# Random Forest Modell erstellen und trainieren mit den Hyperparamteren, die wir zuvor ermittelt haben 
forest_model = RandomForestClassifier(n_estimators=200,max_depth=None,min_samples_split = 2, min_samples_leaf = 1, max_features='log2')
forest_model.fit(X_train, y_train)

# Vorhersagen und Konfusionsmatrix
y_pred = forest_model.predict(X_test)
conf_mat = confusion_matrix(y_test, y_pred)

# Konfusionsmatrix plotten
sns.heatmap(conf_mat, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

print(classification_report(y_test, y_pred))

**Fazit:**

Die Automatisierung des Findens der besten Parameter durch Verfahren wie RandomizedSearchCV bietet eine gute  Möglichkeit, eine breite Palette von Hyperparameter-Kombinationen systematisch zu testen. Es ist jedoch wichtig zu beachten, dass dies nicht immer garantiert, dass man die absolut besten Parameter für jedes Szenario findet. Es gibt mehrere Gründe, warum die Ergebnisse variieren können:

- Trotz der Fortschritte in automatisierten Methoden kann das Verständnis der Daten und des Problems dazu führen, dass einfache oder intuitive Parameterkonfigurationen manchmal ebenso effektiv oder sogar besser sind als die durch automatisierte Suchverfahren gefundenen.

- Experimentieren ist der Schlüssel: Maschinelles Lernen ist stark empirisch geprägt. Auch wenn systematische Suchmethoden einen guten Überblick über mögliche Parameterkombinationen bieten, müssen die als „beste“ identifizierten Parameter nicht in jedem Fall die beste oder optimalste lösung darstellen. Das Durchführen eigener Experimente kann oft zu überraschend positiven Ergebnissen führen. Probier verschiedene Ansätze aus.

Auch Datenbereinigung und die Auswahl geeigneter Features spielen eine große Rolle für die Güte der Vorhersagen. Experimentieren ist Alles!


